In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2021-ai-w7-p2/sample_submission.csv
/kaggle/input/2021-ai-w7-p2/x_test.csv
/kaggle/input/2021-ai-w7-p2/y_train.csv
/kaggle/input/2021-ai-w7-p2/x_train.csv


In [2]:
X = pd.read_csv("/kaggle/input/2021-ai-w7-p2/x_train.csv",engine='python',encoding='CP949')
y = pd.read_csv("/kaggle/input/2021-ai-w7-p2/y_train.csv",engine='python',encoding='CP949')
X_test = pd.read_csv("/kaggle/input/2021-ai-w7-p2/x_test.csv",engine='python',encoding='CP949')
submit = pd.read_csv("/kaggle/input/2021-ai-w7-p2/sample_submission.csv",engine='python',encoding='CP949')

print(X.info())
print(X_test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21486 entries, 0 to 21485
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   시도명       21486 non-null  int64  
 1   읍면동명      21486 non-null  int64  
 2   거주인구      21486 non-null  float64
 3   근무인구      21486 non-null  float64
 4   방문인구      21486 non-null  float64
 5   총 유동인구    21486 non-null  float64
 6   평균 속도     21486 non-null  float64
 7   평균 소요 시간  21486 non-null  float64
 8   평균 기온     21486 non-null  float64
 9   일강수량      21486 non-null  float64
 10  평균 풍속     21486 non-null  float64
 11  월         21486 non-null  int64  
 12  일         21486 non-null  int64  
 13  년         21486 non-null  int64  
dtypes: float64(9), int64(5)
memory usage: 2.3 MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10583 entries, 0 to 10582
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   시도명       10583 non-

In [3]:
X.describe()

,시도명,읍면동명,거주인구,근무인구,방문인구,총 유동인구,평균 속도,평균 소요 시간,평균 기온,일강수량,평균 풍속,월,일,년
count,21486.000000,21486.000000,2.148600e+04,21486.000000,21486.000000,2.148600e+04,21486.000000,21486.000000,21486.000000,21486.000000,21486.000000,21486.000000,21486.000000,21486.000000
mean,0.596435,19.815322,3.173133e+05,35610.667755,196055.537429,5.489795e+05,41.090723,37.192712,13.568007,7.325534,2.746740,6.078563,15.740343,2018.730103
std,0.490624,11.848479,2.986918e+05,40520.154917,140616.040681,4.615707e+05,8.694502,12.959750,7.732429,28.564031,1.483265,3.496635,8.808749,0.704910
min,0.000000,0.000000,9.305552e+03,1172.448000,10788.381000,2.225181e+04,23.545000,12.667000,-11.600000,0.000000,0.000000,1.000000,1.000000,2018.000000
25%,0.000000,10.000000,9.545104e+04,12172.578500,100026.639500,2.228646e+05,34.250000,27.909000,7.600000,0.000000,1.750000,3.000000,8.000000,2018.000000
50%,1.000000,20.000000,2.214727e+05,21871.497000,153199.927500,3.869563e+05,39.622000,34.461000,13.333000,0.000000,2.400000,6.000000,16.000000,2019.000000
75%,1.000000,30.000000,4.079880e+05,40200.021750,235891.613250,6.389216e+05,49.040000,46.071000,19.700000,1.500000,3.400000,9.000000,23.000000,2019.000000
max,1.000000,40.000000,1.353561e+06,265708.771000,786387.113000,2.079278e+06,95.000000,172.200000,30.400000,587.500000,17.700000,12.000000,31.000000,2020.000000


In [4]:
import random
import torch

random.seed(777)
torch.manual_seed(777)
torch.cuda.manual_seed_all(777)

In [5]:
X.columns

Index(['시도명', '읍면동명', '거주인구', '근무인구', '방문인구', '총 유동인구', '평균 속도', '평균 소요 시간',
       '평균 기온', '일강수량', '평균 풍속', '월', '일', '년'],
      dtype='object')

In [6]:
drop_cols = ['읍면동명','월', '일', '년']
X = X.drop(drop_cols, axis = 1)
X_test = X_test.drop(drop_cols, axis = 1)

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y.squeeze(), shuffle = True, random_state = 777, test_size = 0.3)

In [8]:
X_train_tensor = torch.FloatTensor(X_train.to_numpy()).cuda()
X_val_tensor = torch.FloatTensor(X_val.to_numpy()).cuda()

y_train_tensor = torch.FloatTensor(y_train.to_numpy()).cuda()
y_val_tensor = torch.FloatTensor(y_val.to_numpy()).cuda()

X_test_tensor = torch.FloatTensor(X_test.to_numpy()).cuda()

print(X_train_tensor.shape[1])

10


In [9]:
in_features = X_train_tensor.shape[1]
out_features = 1
node = 50

linear1 = torch.nn.Linear(in_features, node)
linear2 = torch.nn.Linear(node, node)
linear3 = torch.nn.Linear(node, node)
linear4 = torch.nn.Linear(node, node)
linear5 = torch.nn.Linear(node, node)
linear6 = torch.nn.Linear(node, node)
linear7 = torch.nn.Linear(node, 1)

torch.nn.init.xavier_normal_(linear1.weight)
torch.nn.init.xavier_normal_(linear2.weight)    
torch.nn.init.xavier_normal_(linear3.weight)    
torch.nn.init.xavier_normal_(linear4.weight)    
torch.nn.init.xavier_normal_(linear5.weight)
torch.nn.init.xavier_normal_(linear6.weight)
torch.nn.init.xavier_normal_(linear7.weight)

relu = torch.nn.ReLU()
dropout = torch.nn.Dropout(p = 0.2)

model = torch.nn.Sequential(linear1, relu, dropout,
                           linear2, relu, dropout,
                           linear3, relu, dropout,
                           linear4, relu, dropout,
                            linear5, relu, dropout,
                            linear6, relu, dropout,
                           linear7).cuda()

In [10]:
from sklearn.metrics import r2_score


nb_epoch = 50000
lr = 0.0001
loss = torch.nn.MSELoss().cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10000, gamma=0.8)
# scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor = 0.5)

model.train()
for epoch in range(nb_epoch+1):
    hypothesis = model(X_train_tensor)
    cost = torch.sqrt(loss(hypothesis, y_train_tensor.unsqueeze(1)))
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
#     scheduler.step()
    
    if epoch%5000 == 0 :
        model.eval()
        acc = r2_score(y_val,model(X_val_tensor).cpu().detach().numpy())
        print(epoch, cost.item(), '{:.10f}'.format(acc))
        model.train()


0 56154.16796875 -4631.8761839257
5000 508.3101806640625 -0.0212153533
10000 458.7123718261719 0.0487556187
15000 426.79071044921875 0.0938850924
20000 413.60003662109375 0.0645229742
25000 408.9442138671875 0.0607600817
30000 404.9692077636719 0.0598826521
35000 402.6430358886719 0.0715119929
40000 395.0987548828125 0.0888417592
45000 387.05474853515625 0.1138575672
50000 376.8623352050781 0.1660922249


In [11]:
with torch.no_grad():
    model.eval()
    y_pred = model(X_test_tensor)
    y_pred = y_pred.cpu().detach().numpy()
    
y_pred

array([[ 84.78177 ],
       [104.79368 ],
       [ 39.609943],
       ...,
       [193.53496 ],
       [176.89925 ],
       [256.0312  ]], dtype=float32)

In [12]:
submit['predict'] = y_pred
submit.to_csv('submission.csv', mode = 'w', index = False)

**********************

In [13]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

X = pd.read_csv("/kaggle/input/2021-ai-w7-p2/x_train.csv",engine='python',encoding='CP949')
y = pd.read_csv("/kaggle/input/2021-ai-w7-p2/y_train.csv",engine='python',encoding='CP949')
X_test = pd.read_csv("/kaggle/input/2021-ai-w7-p2/x_test.csv",engine='python',encoding='CP949')

drop_cols = ['읍면동명','월', '일', '년']
X = X.drop(drop_cols, axis = 1)
X_test = X_test.drop(drop_cols, axis = 1)

sc = StandardScaler()
sc.fit(X)

from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y.squeeze(), shuffle = True, random_state = 777, test_size = 0.3)

X_train_sc = torch.FloatTensor(sc.transform(X_train)).cuda()
X_val_sc = torch.FloatTensor(sc.transform(X_val)).cuda()
X_test_sc = torch.FloatTensor(sc.transform(X_test)).cuda()

y_train_tensor = torch.FloatTensor(y_train.to_numpy()).cuda()
y_val_tensor = torch.FloatTensor(y_val.to_numpy()).cuda()

In [14]:
in_features = X_train_sc.shape[1]
out_features = 1
node = 100

linear1 = torch.nn.Linear(in_features, node)
linear2 = torch.nn.Linear(node, node)
linear3 = torch.nn.Linear(node, node)
linear4 = torch.nn.Linear(node, node)
linear5 = torch.nn.Linear(node, node)
linear6 = torch.nn.Linear(node, node)
linear7 = torch.nn.Linear(node, 1)

torch.nn.init.xavier_normal_(linear1.weight)
torch.nn.init.xavier_normal_(linear2.weight)    
torch.nn.init.xavier_normal_(linear3.weight)    
torch.nn.init.xavier_normal_(linear4.weight)    
torch.nn.init.xavier_normal_(linear5.weight)
torch.nn.init.xavier_normal_(linear6.weight)
torch.nn.init.xavier_normal_(linear7.weight)

relu = torch.nn.ReLU()
dropout = torch.nn.Dropout(p = 0.3)

model = torch.nn.Sequential(linear1, relu, dropout,
                           linear2, relu, dropout,
                           linear3, relu, dropout,
                           linear4, relu, dropout,
                            linear5, relu, dropout,
                            linear6, relu, dropout,
                           linear7).cuda()


In [15]:
from sklearn.metrics import r2_score

nb_epoch = 50000
# lr = 0.1 # 0.8125922304 깊이5
# lr = 0.01 # 0.9002982957 깊이5
lr = 0.001 
loss = torch.nn.MSELoss().cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1000, gamma=0.5)

model.train()
for epoch in range(nb_epoch+1):
    hypothesis = model(X_train_sc)
    cost = torch.sqrt(loss(hypothesis, y_train_tensor.unsqueeze(1)))
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
#     scheduler.step()
    
    if epoch%5000 == 0 :
        model.eval()
        acc = r2_score(y_val,model(X_val_sc).cpu().detach().numpy())
        print(epoch, cost.item(), '{:.10f}'.format(acc))
        model.train()
    

0 648.2200927734375 -0.5046156718
5000 135.90338134765625 0.9469343520
10000 132.45440673828125 0.9503405673
15000 125.63963317871094 0.9531490716
20000 123.4761962890625 0.9543487333
25000 124.51055145263672 0.9548704079
30000 121.18698120117188 0.9550659731
35000 123.1278305053711 0.9513418518
40000 119.39875793457031 0.9518950409
45000 120.50129699707031 0.9496180968
50000 116.65650177001953 0.9522245420


In [16]:
with torch.no_grad():
    model.eval()
    y_pred = model(X_test_sc)
    y_pred = y_pred.cpu().detach().numpy()
    
y_pred

array([[ 57.287895],
       [ 55.124195],
       [ 54.480843],
       ...,
       [ 54.068478],
       [ 54.03348 ],
       [751.8519  ]], dtype=float32)

In [17]:
submit['predict'] = y_pred
submit.to_csv('submission_sc.csv', mode = 'w', index = False)